In [1]:
name_list = ['Carlsen, Magnus',
 'Ding, Liren',
 'Nepomniachtchi, Ian',
 'Firouzja, Alireza',
 'Nakamura, Hikaru',
 'Caruana, Fabiano',
 'Giri, Anish',
 'So, Wesley',
 'Anand, Viswanathan',
 'Karjakin, Sergey',
 'Radjabov, Teimour',
 'Grischuk, Alexander',
 'Dominguez Perez, Leinier',
 'Mamedyarov, Shakhriyar',
 'Rapport, Richard',
 'Vachier-Lagrave, Maxime',
 'Aronian, Levon',
 'Vidit, Santosh Gujrathi',
 'Yu, Yangyi',
 'Andreikin, Dmitry',
 'Duda, Jan Krzysztof',
 'Le, Quang Liem',
 'Topalov, Veselin',
 'Gukesh D',
 'Vitiugov, Nikita',
 'Erigaisi Arjun',
 'Wang, Hao',
 'Wei, Yi',
 'Maghsoodloo, Parham',
 'Abdusattorov, Nodirbek',
 'Sjugirov, Sanan',
 'Vallejo Pons, Francisco',
 'Shankland, Samuel L',
 'Dubov, Daniil',
 'Niemann, Hans Moke',
 'Eljanov, Pavel',
 'Harikrishna, Pentala',
 'Robson, Ray',
 'Artemiev, Vladislav',
 'Deac, Bogdan-Daniel',
 'Sargissian, Gabriel',
 'Bu, Xiangzhi',
 'Keymer, Vincent',
 'Tomashevsky, Evgeny',
 'Xiong, Jeffery',
 'Adams, Michael',
 'Sevian, Samuel',
 'Alekseenko, Kirill',
 'Tabatabaei, M. Amin',
 'Wojtaszek, Radoslaw',
 'Amin, Bassem',
 'Praggnanandhaa R',
 'Predke, Alexandr',
 'Svidler, Peter',
 'Navara, David',
 'Jakovenko, Dmitrij',
 'Martirosyan, Haik M.',
 'Moussard, Jules',
 'Van Foreest, Jorden',
 'Anton Guijarro, David',
 'Kasimdzhanov, Rustam',
 'Almasi, Zoltan',
 'Salem, A.R. Saleh',
 'Esipenko, Andrey',
 'Fedoseev, Vladimir',
 'Kovalenko, Igor',
 'Li, Chao B',
 'Oparin, Grigoriy',
 'Nihal Sarin',
 'Narayanan.S.L',
 'Sarana, Alexey',
 'Gelfand, Boris',
 'Ni, Hua',
 'Shirov, Alexei',
 'Kamsky, Gata',
 'Volokitin, Andrei',
 'Ivanchuk, Vassily',
 'Howell, David',
 'Naiditsch, Arkadij',
 'Shevchenko, Kirill',
 'Matlakov, Maxim',
 'Najer, Evgeniy',
 'Bluebaum, Matthias',
 'Guseinov, Gadir',
 'Inarkiev, Ernesto',
 'Grandelius, Nils',
 'Cheparinov, Ivan',
 'Ma, Qun',
 'Saric, Ivan',
 'Santos Latasa, Jaime',
 'Nguyen, Thai Dai Van',
 'Ponomariov, Ruslan',
 'Nyzhnyk, Illya',
 'Sindarov, Javokhir',
 'Tari, Aryan',
 'Bacrot, Etienne',
 'Cori, Jorge',
 'Korobov, Anton',
 'Malakhov, Vladimir',
 'Kryvoruchko, Yuriy']

In [4]:
import re
import pandas as pd
from urllib.request import Request, urlopen
from tqdm import tqdm

class Chess_Elo(object):
    def __init__(self, name):
        self.name = name
        self.n = 0

    def tqdm_generator(self):
        while True:
            yield

    def chess_result_player(self, color, result):
        game_result = result.split('-')
        if color == 'white':
            if game_result[0] == '1':
                return 'win'
            elif game_result[0] == '0':
                return 'lose'
            else:
                return 'draw'
        else:
            if game_result[1] == '1':
                return 'win'
            elif game_result[1] == '0':
                return 'lose'
            else:
                return 'draw'

    def scrape_chess_elo(self):

        df_temp = pd.DataFrame(columns=['game_id', 
                                        'white_player', 
                                        'white_player_rating',
                                        'black_player',
                                        'black_player_rating',
                                        'game_result',
                                        'move',
                                        'ECO',
                                        'site',
                                        'year'])

        for _ in tqdm(self.tqdm_generator()):
            self.n += 1
            begin = len(df_temp)
            name_url = self.name
            name_url = name_url.split()
            name_url = '%20'.join(name_url)
            
            url=f"https://2700chess.com/games?search={name_url}&page={self.n}"
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

            web_byte = urlopen(req).read()

            webpage = web_byte.decode('utf-8')
                    
            result = re.findall('tr data-key(.*)tr', webpage)
                    
            for i in result:
                lst = []
                for k in i.split('><'):
                    try:
                        val_temp = re.search(r'>(.*)<', k).group(1)
                        if val_temp == ' ':
                            lst.append(None)
                        else:
                            lst.append(val_temp)
                    except: 
                        pass
                try:
                    df_temp.loc[len(df_temp)] = lst
                except:
                    try:
                        if bool(re.search('^[A-Z]{1}[0-9]{2}$', lst[7])):
                            lst.insert(8, None)
                            df_temp.loc[len(df_temp)] = lst
                        else:
                            lst.insert(7, None)
                            df_temp.loc[len(df_temp)] = lst
                    except:
                        lst.insert(7, None)
                        df_temp.loc[len(df_temp)] = lst
            
            if begin == len(df_temp):
                break
            else:
                pass

        return df_temp

    def play_as_decider(self, player_name_dependent ,player_name_control):
        try:
            if player_name_dependent == player_name_control:
                return 'white'
            else:
                return 'black'
        except:
            return 'black'

    def df_elo(self):
        df = self.scrape_chess_elo().copy()
        player_name = df['white_player'].mode()

        df['play_as'] = df['white_player'].apply(lambda x: self.play_as_decider(x, player_name[0]))
        df['rating'] = df.apply(lambda x: x['white_player_rating'] if x['play_as'] == 'white' else x['black_player_rating'], axis=1)
        df['opponent'] = df.apply(lambda x: x['white_player'] if x['play_as'] == 'black' else x['black_player'], axis=1)
        df['opponent_rating'] = df.apply(lambda x: x['black_player_rating'] if x['play_as'] == 'white' else x['white_player_rating'], axis=1)
        df['result'] = df.apply(lambda x: self.chess_result_player(x['play_as'], x['game_result']), axis=1)
        df[['rating', 'play_as', 'opponent', 'opponent_rating', 'result', 'move', 'ECO', 'site', 'year']].to_csv(f'data/{self.name}.csv', index=False)

In [5]:
# name index 60 has a problem


for name in tqdm(['Howell, David']):
    try:
        temp = Chess_Elo(name)
        temp.df_elo()
    except:
        error_list.append(name)

68it [00:30,  2.23it/s]00:00<?, ?it/s]
31it [00:12,  2.40it/s]00:30<27:43, 30.81s/it]
27it [00:21,  1.23it/s]00:43<17:58, 20.35s/it]
4it [00:04,  1.02s/it][01:05<18:19, 21.14s/it]
50it [00:41,  1.20it/s]01:10<12:15, 14.43s/it]
30it [00:23,  1.26it/s]01:51<20:13, 24.26s/it]
5it [00:03,  1.30it/s][02:15<19:41, 24.12s/it]
24it [00:19,  1.26it/s]02:19<14:00, 17.50s/it]
73it [01:01,  1.18it/s]02:38<14:07, 18.04s/it]
60it [00:53,  1.12it/s]03:40<24:19, 31.73s/it]
1it [00:00,  1.54it/s] [04:34<28:56, 38.59s/it]
5it [00:03,  1.44it/s] [04:35<19:47, 26.98s/it]
20it [00:15,  1.27it/s][04:38<14:13, 19.84s/it]
43it [00:38,  1.12it/s][04:54<13:02, 18.64s/it]
36it [00:28,  1.27it/s][05:32<16:48, 24.61s/it]
36it [00:29,  1.21it/s][06:01<17:11, 25.79s/it]
43it [00:37,  1.14it/s][06:31<17:34, 27.05s/it]
27it [00:25,  1.06it/s][07:09<19:12, 30.34s/it]
35it [00:31,  1.11it/s][07:34<17:48, 28.89s/it]
34it [00:27,  1.22it/s][08:06<17:50, 29.74s/it]
38it [00:31,  1.21it/s][08:34<17:03, 29.23s/it]
27it [00:2

In [71]:
import glob
for f in glob.iglob("*.csv"): # generator, search immediate subdirectories 
    temp = pd.read_csv(f)
    if temp.shape[0] < 300:
        print(f.replace('.csv', ''), ':' ,temp.shape[0])
    else:
        pass

Duda, Jan-Krzysztof : 143
Howell, David W L : 1
Ivanchuk, Vasyl : 53
Jakovenko, Dmitry : 2
Martirosyan, Haik M. : 207
Narayanan.S.L : 164
Praggnanandhaa R : 219
Shankland, Sam : 27
Tabatabaei, M. Amin : 179


In [72]:
new_list_name = ['Duda, Jan Krzysztof',
                 'Howell, David',
                 'Ivanchuk, Vassily',
                 'Jakovenko, Dmitrij',
                 'Martirosyan, Haik M.',
                 'Narayanan.S.L',
                 'Praggnanandhaa R',
                 'Shankland, Samuel L',
                 'Tabatabaei, M. Amin']

In [74]:
# name index 60 has a problem
for name in tqdm(new_list_name):
    try:
        temp = Chess_Elo(name)
        temp.df_elo()
    except:
        error_list.append(name)

31it [00:12,  2.44it/s]0:00<?, ?it/s]
41it [00:33,  1.23it/s]0:12<01:42, 12.80s/it]
86it [01:20,  1.07it/s]0:46<02:54, 24.89s/it]
42it [00:32,  1.28it/s]2:06<05:00, 50.16s/it]
5it [00:03,  1.28it/s]02:39<03:36, 43.40s/it]
4it [00:03,  1.27it/s]02:43<01:56, 29.18s/it]
5it [00:04,  1.14it/s]02:46<01:01, 20.34s/it]
27it [00:22,  1.18it/s]2:50<00:30, 15.14s/it]
4it [00:03,  1.15it/s]03:14<00:17, 17.66s/it]
100%|██████████| 9/9 [03:17<00:00, 21.95s/it]


In [ ]:
error_list_2

## Magnus

In [1]:
'%2C+'.join(['Magnus', 'Carlsen'])

'Magnus%2C+Carlsen'

In [30]:
df_magnus = pd.DataFrame(columns=['game_id', 
                                  'white_player', 
                                  'white_player_rating',
                                  'black_player',
                                  'black_player_rating',
                                  'game_result',
                                  'move',
                                  'ECO',
                                  'site',
                                  'year'])



n = 0

while True:
    n += 1
    begin = len(df_magnus)
    url=f"https://2700chess.com/games?search=Carlsen%2C+Magnus&page={n}"
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    web_byte = urlopen(req).read()

    webpage = web_byte.decode('utf-8')
            
    result = re.findall('tr data-key(.*)tr', webpage)
            
    for i in result:
        lst = []
        for k in i.split('><'):
            try:
                val_temp = re.search(r'>(.*)<', k).group(1)
                if val_temp == ' ':
                    lst.append(None)
                else:
                    lst.append(val_temp)
            except: 
                pass
        try:
            df_magnus.loc[len(df_magnus)] = lst
        except:
            lst.insert(7, None)
            df_magnus.loc[len(df_magnus)] = lst
    
    if begin == len(df_magnus):
        break
    else:
        pass


df_magnus_csv = df_magnus.copy()

df_magnus_csv['play_as'] = df_magnus_csv['white_player'].apply(lambda x: 'white' if x == 'Carlsen, Magnus' else 'black')
df_magnus_csv['rating'] = df_magnus_csv.apply(lambda x: x['white_player_rating'] if x['play_as'] == 'white' else x['black_player_rating'], axis=1)
df_magnus_csv['opponent'] = df_magnus_csv.apply(lambda x: x['white_player'] if x['play_as'] == 'black' else x['black_player'], axis=1)
df_magnus_csv['opponent_rating'] = df_magnus_csv.apply(lambda x: x['black_player_rating'] if x['play_as'] == 'white' else x['white_player_rating'], axis=1)
df_magnus_csv['result'] = df_magnus_csv.apply(lambda x: chess_result_player(x['play_as'], x['game_result']), axis=1)



KeyboardInterrupt: 

In [28]:
df_magnus_csv

NameError: name 'df_magnus_csv' is not defined

In [135]:
df_magnus_csv = df_magnus[:-3].copy()

In [137]:
df_magnus_csv['play_as'] = df_magnus_csv['white_player'].apply(lambda x: 'white' if x == 'Carlsen, Magnus' else 'black')
df_magnus_csv['rating'] = df_magnus_csv.apply(lambda x: x['white_player_rating'] if x['play_as'] == 'white' else x['black_player_rating'], axis=1)
df_magnus_csv['opponent'] = df_magnus_csv.apply(lambda x: x['white_player'] if x['play_as'] == 'black' else x['black_player'], axis=1)
df_magnus_csv['opponent_rating'] = df_magnus_csv.apply(lambda x: x['black_player_rating'] if x['play_as'] == 'white' else x['white_player_rating'], axis=1)
df_magnus_csv['result'] = df_magnus_csv.apply(lambda x: chess_result_player(x['play_as'], x['game_result']), axis=1)



In [138]:
df_magnus_csv[['rating', 'play_as', 'opponent', 'opponent_rating', 'result', 'move', 'ECO', 'site', 'year']].head(3)

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
0,2859,black,"Giri, Anish",2764,lose,35,E15,Wijk aan Zee,2023
1,2859,white,"Ding, Liren",2811,draw,37,A46,Wijk aan Zee,2023
2,2859,black,"Keymer, Vincent",2696,win,65,D91,Wijk aan Zee,2023


In [139]:
df_magnus_csv[['rating', 'play_as', 'opponent', 'opponent_rating', 'result', 'move', 'ECO', 'site', 'year']].to_csv('magnus.csv', index=False)

In [140]:
df_test = pd.read_csv('magnus.csv')
df_test.head()

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
0,2859.0,black,"Giri, Anish",2764.0,lose,35,E15,Wijk aan Zee,2023
1,2859.0,white,"Ding, Liren",2811.0,draw,37,A46,Wijk aan Zee,2023
2,2859.0,black,"Keymer, Vincent",2696.0,win,65,D91,Wijk aan Zee,2023
3,2859.0,white,"Aronian, Levon",2735.0,draw,41,E10,Wijk aan Zee,2023
4,2830.0,white,"Abdusattorov, Nodirbek",2666.0,win,38,C77,Almaty,2022


In [141]:
df_test.tail()

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
3989,NaN,black,"Skovgaard, Peter Nicolai",2130.0,win,37,D94,Norway,2001
3990,NaN,black,"Bartels, Holger",2170.0,draw,48,C59,Bad Wiessee,2000
3991,NaN,white,"Olzem, Lothar",2179.0,draw,36,D00,Bad Wiessee,2000
3992,NaN,black,"Kaiser, Guenter",NaN,win,36,B08,Bad Wiessee,2000
3993,NaN,white,"Cordts, Ingo",2222.0,lose,30,A31,Bad Wiessee,2000


## Kasparov


In [142]:
df_garry = pd.DataFrame(columns=['game_id', 
                                  'white_player', 
                                  'white_player_rating',
                                  'black_player',
                                  'black_player_rating',
                                  'game_result',
                                  'move',
                                  'ECO',
                                  'site',
                                  'year'])

n = 0

while True:
    n += 1
    begin = len(df_garry)
    url=f"https://2700chess.com/games?search=Kasparov,%20Garry&page={n}"
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    web_byte = urlopen(req).read()

    webpage = web_byte.decode('utf-8')
            
    result = re.findall('tr data-key(.*)tr', webpage)
            
    for i in result:
        lst = []
        for k in i.split('><'):
            try:
                val_temp = re.search(r'>(.*)<', k).group(1)
                if val_temp == ' ':
                    lst.append(None)
                else:
                    lst.append(val_temp)
            except: 
                pass
        try:
            df_garry.loc[len(df_garry)] = lst
        except:
            lst.insert(7, None)
            df_garry.loc[len(df_garry)] = lst
    
    if begin == len(df_garry):
        break
    else:
        pass
    

In [143]:
df_garry

,game_id,white_player,white_player_rating,black_player,black_player_rating,game_result,move,ECO,site,year
0,1,"Nepomniachtchi, Ian",2807,"Kasparov, Garry",2801,1/2-1/2,30,A10,Zagreb,2021
1,2,"Kasparov, Garry",2801,"Vachier-Lagrave, Maxime",2794,0-1,43,A04,Zagreb,2021
2,3,"Giri, Anish",2744,"Kasparov, Garry",2801,1/2-1/2,39,A04,Zagreb,2021
3,4,"Kasparov, Garry",2801,"Mamedyarov, Shakhriyar",2716,0-1,26,D61,Zagreb,2021
4,5,"Korobov, Anton",2668,"Kasparov, Garry",2801,1-0,17,E73,Zagreb,2021
...,...,...,...,...,...,...,...,...,...,...
2577,2578,"Kasparov, Garry",None,"Averbakh, Yuri L",None,1-0,48,C68,Moscow,1974
2578,2579,"Magerramov, Elmar",None,"Kasparov, Garry",None,0-1,35,B84,Baku,1973
2579,2580,"Kengis, Edvins",None,"Kasparov, Garry",None,1/2-1/2,54,B89,Vilnius,1973
2580,2581,"Kasparov, Garry",None,"Muratkuliev, Shohrat",None,1-0,32,C84,Baku,1973


In [154]:
df_garry_csv = df_garry.copy()

In [155]:
def chess_result_player(color, game_result):
    game_result = game_result.split('-')
    if color == 'white':
        if game_result[0] == '1':
            return 'win'
        elif game_result[0] == '0':
            return 'lose'
        else:
            return 'draw'
    else:
        if game_result[1] == '1':
            return 'win'
        elif game_result[1] == '0':
            return 'lose'
        else:
            return 'draw'

In [159]:
df_garry_csv['play_as'] = df_garry_csv['white_player'].apply(lambda x: 'white' if x == 'Kasparov, Garry' else 'black')
df_garry_csv['rating'] = df_garry_csv.apply(lambda x: x['white_player_rating'] if x['play_as'] == 'white' else x['black_player_rating'], axis=1)
df_garry_csv['opponent'] = df_garry_csv.apply(lambda x: x['white_player'] if x['play_as'] == 'black' else x['black_player'], axis=1)
df_garry_csv['opponent_rating'] = df_garry_csv.apply(lambda x: x['black_player_rating'] if x['play_as'] == 'white' else x['white_player_rating'], axis=1)
df_garry_csv['result'] = df_garry_csv.apply(lambda x: chess_result_player(x['play_as'], x['game_result']), axis=1)



In [168]:
df_garry_csv[['rating', 'play_as', 'opponent', 'opponent_rating', 'result', 'move', 'ECO', 'site', 'year']].head(5)

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
0,2801,black,"Nepomniachtchi, Ian",2807,draw,30,A10,Zagreb,2021
1,2801,white,"Vachier-Lagrave, Maxime",2794,lose,43,A04,Zagreb,2021
2,2801,black,"Giri, Anish",2744,draw,39,A04,Zagreb,2021
3,2801,white,"Mamedyarov, Shakhriyar",2716,lose,26,D61,Zagreb,2021
4,2801,black,"Korobov, Anton",2668,lose,17,E73,Zagreb,2021


In [169]:
df_garry_csv[['rating', 'play_as', 'opponent', 'opponent_rating', 'result', 'move', 'ECO', 'site', 'year']].to_csv('garry.csv', index=False)

In [170]:
df_test = pd.read_csv('garry.csv')
df_test.head()

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
0,2801.0,black,"Nepomniachtchi, Ian",2807.0,draw,30,A10,Zagreb,2021
1,2801.0,white,"Vachier-Lagrave, Maxime",2794.0,lose,43,A04,Zagreb,2021
2,2801.0,black,"Giri, Anish",2744.0,draw,39,A04,Zagreb,2021
3,2801.0,white,"Mamedyarov, Shakhriyar",2716.0,lose,26,D61,Zagreb,2021
4,2801.0,black,"Korobov, Anton",2668.0,lose,17,E73,Zagreb,2021


In [171]:
df_test.tail()

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
2577,NaN,white,"Averbakh, Yuri L",NaN,win,48,C68,Moscow,1974
2578,NaN,black,"Magerramov, Elmar",NaN,win,35,B84,Baku,1973
2579,NaN,black,"Kengis, Edvins",NaN,draw,54,B89,Vilnius,1973
2580,NaN,white,"Muratkuliev, Shohrat",NaN,win,32,C84,Baku,1973
2581,NaN,white,Vasilienko,NaN,win,40,C04,Kiev,1973
